<a href="https://colab.research.google.com/github/drewpager/deep-learning/blob/main/11_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import string

class Vectorizer:
  def standardize(self, text):
    text = text.lower()
    return "".join(char for char in text if char not in string.punctuation)

  def tokenize(self, text):
    text = self.standardize(text)
    return text.split()
  
  def make_vocabulary(self, dataset):
    self.vocabulary = {"": 0, "[UNK]": 1}
    for text in dataset:
      text = self.standardize(text)
      tokens = self.tokenize(text)
      for token in tokens:
        if token not in self.vocabulary:
          self.vocabulary[token] = len(self.vocabulary)
          self.inverse_vocabulary = dict((v, k) for k, v in self.vocabulary.items())
  
  def encode(self, text):
    text = self.standardize(text)
    tokens = self.tokenize(text)
    return [self.vocabulary.get(token, 1) for token in tokens]

  def decode(self, int_sequence):
    return " ".join(self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

vectorizer = Vectorizer()
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",           
]

vectorizer.make_vocabulary(dataset)



In [27]:
test_sentence = "I write, rewrite, and still rewrite again."
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

[2, 3, 5, 7, 1, 5, 6]
i write rewrite and [UNK] rewrite again


In [46]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

text_vectorization = layers.TextVectorization(output_mode="int")
text_vectorization.adapt(dataset)
vocabulary = text_vectorization.get_vocabulary()
encode_sent = text_vectorization(test_sentence)
print(encode_sent)
inverse_sent = dict(enumerate(vocabulary))
decode_sent = " ".join(inverse_sent[int(i)] for i in encode_sent)
print(decode_sent)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)
i write rewrite and [UNK] rewrite again


**IMBD Review Sentiment Analysis**

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  48.9M      0  0:00:01  0:00:01 --:--:-- 48.9M


In [3]:
!rm -r aclImdb/train/unsup

In [4]:
!cat /content/aclImdb/train/pos/10009_9.txt

When I first read Armistead Maupins story I was taken in by the human drama displayed by Gabriel No one and those he cares about and loves. That being said, we have now been given the film version of an excellent story and are expected to see past the gloss of Hollywood...<br /><br />Writer Armistead Maupin and director Patrick Stettner have truly succeeded! <br /><br />With just the right amount of restraint Robin Williams captures the fragile essence of Gabriel and lets us see his struggle with issues of trust both in his personnel life(Jess) and the world around him(Donna).<br /><br />As we are introduced to the players in this drama we are reminded that nothing is ever as it seems and that the smallest event can change our lives irrevocably. The request to review a book written by a young man turns into a life changing event that helps Gabriel find the strength within himself to carry on and move forward.<br /><br />It's to bad that most people will avoid this film. I only say that

In [8]:
import os, shutil, pathlib, random 

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("pos", "neg"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)


In [11]:
from tensorflow import keras
batch_size = 32

train_ds = keras.preprocessing.text_dataset_from_directory("/content/aclImdb/train", batch_size=batch_size)
val_ds = keras.preprocessing.text_dataset_from_directory("/content/aclImdb/val", batch_size=batch_size)
test_ds = keras.preprocessing.text_dataset_from_directory("/content/aclImdb/test",batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
